### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('prices.csv')


### Check all columns in the dataset

In [5]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [6]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [7]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [8]:
data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set

In [9]:
data = data.iloc[:1000,:]

In [10]:
data.shape

(1000, 5)

In [11]:
feature_set = data[['open','low','high','volume']]
target_set =data['close']

### Divide the data into train and test sets

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(feature_set, target_set, test_size=0.3, random_state=1)


## Building the graph in tensorflow

1.Define input data placeholders

In [13]:
#input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

x_n = tf.nn.l2_normalize(x,1)

#0utput features
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [14]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

Instructions for updating:
Colocations handled automatically by placer.


3.Prediction

In [15]:
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [16]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [17]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [18]:
sess = tf.Session()


sess.run(tf.global_variables_initializer())
         
training_epochs = 100


In [19]:
for epoch in range(training_epochs):
            
    _, train_loss = sess.run([train_op,loss],feed_dict={x:feature_set, y_:target_set})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  7861.588
Training loss at step:  10  is  3748.3845
Training loss at step:  20  is  3429.3552
Training loss at step:  30  is  3404.6091
Training loss at step:  40  is  3402.6887
Training loss at step:  50  is  3402.5408
Training loss at step:  60  is  3402.5308
Training loss at step:  70  is  3402.5269
Training loss at step:  80  is  3402.5269
Training loss at step:  90  is  3402.5288


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [20]:
sess.run(W)

array([[2.5700359e-03],
       [2.5463821e-03],
       [2.5909436e-03],
       [3.3387997e+01]], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [21]:
for epoch in range(training_epochs):
            
    _, train_loss1 = sess.run([train_op,loss],feed_dict={x:X_train, y_:Y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss1)

Training loss at step:  0  is  3567.247
Training loss at step:  10  is  3567.2168
Training loss at step:  20  is  3567.2126
Training loss at step:  30  is  3567.2136
Training loss at step:  40  is  3567.2131
Training loss at step:  50  is  3567.2148
Training loss at step:  60  is  3567.2136
Training loss at step:  70  is  3567.2136
Training loss at step:  80  is  3567.2136
Training loss at step:  90  is  3567.2136


In [25]:
for epoch in range(training_epochs):
            
    
    _, train_loss2 = sess.run([train_op,loss],feed_dict={x:X_test, y_:Y_test})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss2)

Training loss at step:  0  is  3017.9866
Training loss at step:  10  is  3017.9866
Training loss at step:  20  is  3017.9866
Training loss at step:  30  is  3017.9866
Training loss at step:  40  is  3017.9866
Training loss at step:  50  is  3017.9866
Training loss at step:  60  is  3017.9866
Training loss at step:  70  is  3017.9866
Training loss at step:  80  is  3017.9866
Training loss at step:  90  is  3017.9866


In [28]:
difference = abs(train_loss1 - train_loss2)
difference

549.22705

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [35]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

model.add(tf.keras.layers.Dense(1,activation='softmax'))

model.compile(optimizer='sgd', loss='mse')


### Execute the model

In [36]:
model.fit(feature_set, target_set, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 1s 554us/sample - loss: 7729.0365
Epoch 2/5
1000/1000 [==============================] - 0s 31us/sample - loss: 7729.0366
Epoch 3/5
1000/1000 [==============================] - 0s 48us/sample - loss: 7729.0366
Epoch 4/5
1000/1000 [==============================] - 0s 23us/sample - loss: 7729.0366
Epoch 5/5
1000/1000 [==============================] - 0s 47us/sample - loss: 7729.0365


### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [75]:


df = pd.read_csv('iris.csv')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [47]:
df = df.drop(['Id'],axis=1)

### Splitting the data into feature set and target set

In [49]:
feature_set = df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
target_set = df['Species']

In [64]:
feature_set.shape

(150, 4)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [60]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

df['Species'] = labelencoder.fit_transform(target_set)

target = pd.get_dummies(df['Species'])

df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [65]:
target.shape

(150, 3)

### Divide the dataset into Training and test (70:30)

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(feature_set, target, test_size=0.3, random_state=1)


In [70]:
X_train.shape

(105, 4)

In [71]:
Y_train.shape

(105, 3)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [95]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(8))

model.add(tf.keras.layers.Dense(3,activation="softmax"))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [105]:
model.fit(X_train,Y_train, epochs=10,validation_data=(X_test,Y_test))

Train on 105 samples, validate on 45 samples
Epoch 1/10
105/105 [==============================] - 0s 0s/sample - loss: 0.3682 - acc: 0.8762 - val_loss: 0.9798 - val_acc: 0.6000
Epoch 2/10
105/105 [==============================] - 0s 0s/sample - loss: 0.3730 - acc: 0.8762 - val_loss: 0.9932 - val_acc: 0.6000
Epoch 3/10
105/105 [==============================] - 0s 148us/sample - loss: 0.3602 - acc: 0.8667 - val_loss: 0.9852 - val_acc: 0.6000
Epoch 4/10
105/105 [==============================] - 0s 149us/sample - loss: 0.3635 - acc: 0.8476 - val_loss: 0.9725 - val_acc: 0.6000
Epoch 5/10
105/105 [==============================] - 0s 0s/sample - loss: 0.3521 - acc: 0.8762 - val_loss: 0.9556 - val_acc: 0.6000
Epoch 6/10
105/105 [==============================] - 0s 148us/sample - loss: 0.3423 - acc: 0.8762 - val_loss: 0.9451 - val_acc: 0.6000
Epoch 7/10
105/105 [==============================] - 0s 115us/sample - loss: 0.3339 - acc: 0.8857 - val_loss: 0.9221 - val_acc: 0.6000
Epoch 8/10
1

In [101]:
model.predict(X_test)

array([[9.8501575e-01, 1.7453361e-03, 1.3238880e-02],
       [1.8586263e-01, 1.3815467e-01, 6.7598265e-01],
       [3.5203841e-02, 6.4006902e-02, 9.0078926e-01],
       [9.7601795e-01, 2.7961675e-03, 2.1185940e-02],
       [2.4424659e-03, 1.7601846e-02, 9.7995573e-01],
       [2.8464878e-02, 5.2729718e-02, 9.1880542e-01],
       [4.3211482e-03, 1.8952064e-02, 9.7672677e-01],
       [9.4113213e-01, 9.5987264e-03, 4.9269162e-02],
       [9.2949098e-01, 2.0419654e-02, 5.0089356e-02],
       [2.2044047e-03, 2.1457119e-02, 9.7633845e-01],
       [4.0305380e-02, 1.2678975e-01, 8.3290488e-01],
       [9.5904958e-01, 5.3060865e-03, 3.5644382e-02],
       [1.6420725e-03, 3.0459918e-02, 9.6789795e-01],
       [3.6445059e-02, 5.6890741e-02, 9.0666425e-01],
       [2.6016496e-02, 8.1246510e-02, 8.9273691e-01],
       [9.5364851e-01, 1.4028718e-02, 3.2322757e-02],
       [5.0452445e-02, 1.2967657e-01, 8.1987095e-01],
       [3.1633843e-02, 9.2424169e-02, 8.7594199e-01],
       [9.3825585e-01, 9.659

### Report Accuracy of the predicted values

In [102]:
from sklearn.metrics import accuracy_score

In [104]:
scores = model.evaluate(X_test,Y_test)
print(scores)

45/45 [==============================] - 0s 0s/sample - loss: 0.9645 - acc: 0.6000
[0.9645431545045641, 0.6]
